In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
from pydub import AudioSegment
import pandas as pd
import IPython.display as ipd

In [96]:
FOLDER = '/home/usuario/birds/birdsong-recognition/'
TRAIN_FOLDER = FOLDER + 'train_audio/'

train = pd.read_csv(FOLDER+'train.csv')

In [106]:
def get_train_clip(dataframe, resample=None):
    filename = TRAIN_FOLDER+dataframe['ebird_code']+'/'+dataframe['filename']
    sound = AudioSegment.from_mp3(filename)
    orig_sr = sound.frame_rate
    if resample is not None:
        sound = sound.set_frame_rate(resample)
    clip = sound.get_array_of_samples()
    duration = sound.duration_seconds

    clip = np.array(clip)
    clip = (clip - clip.mean())/np.abs(clip).std()
    return clip, orig_sr, duration

def get_spectrum(x):
    X = librosa.stft(x, )
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(14, 5))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    return Xdb

In [107]:
birds_codes = train['ebird_code'].unique()
print(birds_codes[:100])

['aldfly' 'ameavo' 'amebit' 'amecro' 'amegfi' 'amekes' 'amepip' 'amered'
 'amerob' 'amewig' 'amewoo' 'amtspa' 'annhum' 'astfly' 'baisan' 'baleag'
 'balori' 'banswa' 'barswa' 'bawwar' 'belkin1' 'belspa2' 'bewwre' 'bkbcuc'
 'bkbmag1' 'bkbwar' 'bkcchi' 'bkchum' 'bkhgro' 'bkpwar' 'bktspa' 'blkpho'
 'blugrb1' 'blujay' 'bnhcow' 'boboli' 'bongul' 'brdowl' 'brebla' 'brespa'
 'brncre' 'brnthr' 'brthum' 'brwhaw' 'btbwar' 'btnwar' 'btywar' 'buffle'
 'buggna' 'buhvir' 'bulori' 'bushti' 'buwtea' 'buwwar' 'cacwre' 'calgul'
 'calqua' 'camwar' 'cangoo' 'canwar' 'canwre' 'carwre' 'casfin' 'caster1'
 'casvir' 'cedwax' 'chispa' 'chiswi' 'chswar' 'chukar' 'clanut' 'cliswa'
 'comgol' 'comgra' 'comloo' 'commer' 'comnig' 'comrav' 'comred' 'comter'
 'comyel' 'coohaw' 'coshum' 'cowscj1' 'daejun' 'doccor' 'dowwoo' 'dusfly'
 'eargre' 'easblu' 'easkin' 'easmea' 'easpho' 'eastow' 'eawpew' 'eucdov'
 'eursta' 'evegro' 'fiespa' 'fiscro']


In [108]:
import os

In [199]:
def get_bird_data(dataset_folder, ebird_code, target_sr = 22050, chunk_seconds=2, rating_thres = 4, std_thres=0.1, save=True):
    under_tres = []
    if not os.path.exists(dataset_folder):
        os.makedirs(dataset_folder)
    total_duration = 0
    df_aldfly = train[train['ebird_code']==ebird_code]
    chunk_size = chunk_seconds*target_sr
    ebird_folder = dataset_folder + ebird_code + '/'
    if not os.path.exists(ebird_folder):
        os.makedirs(ebird_folder)
    for i in range(len(df_aldfly)):
        df_row = df_aldfly.iloc[i]
        rating = df_row['rating']
        if rating>=rating_thres:
            dataframe = df_aldfly.iloc[i]
            x, orig_sr, duration = get_train_clip(dataframe, target_sr)
            print(f'\r{i}, {orig_sr}, {duration}', end='')
#             print()
#             print(x.mean(), x.std())
#             print()
            x = x[:(len(x)//chunk_size)*chunk_size].reshape(-1, chunk_size)
            
            total_duration = total_duration + duration
            
            for j, row in enumerate(x):
                row_std = row.std()
                
                if row_std > std_thres:
                    # print(j, row.mean(), row.std())
                    if save:
                        file_to_save = ebird_folder + ''.join(dataframe['filename'].split('.')[:-1]) + f'_{j+1}.npy'
                        if not os.path.exists(file_to_save):
                            np.save(file_to_save, row)

                else:
                    under_tres.append(row)
    return under_tres

In [ ]:
dataset_folder = './dataset/all/'
sr = 22050
chunk_seconds = 1
_ = get_bird_data(dataset_folder, 'amecro', target_sr = 22050, chunk_seconds=chunk_seconds)
_ = get_bird_data(dataset_folder, 'aldfly', target_sr = 22050, chunk_seconds=chunk_seconds)
_ = get_bird_data(dataset_folder, 'amegfi', target_sr = 22050, chunk_seconds=chunk_seconds)

26, 44100, 13.897142857142857

In [ ]:
# stds = [x.std() for x in X_amecro]

In [ ]:
# indxs = np.argsort(stds)
# N = 200
# print(indxs[:N])
# print(np.array(stds)[indxs][:N])

In [ ]:
# ipd.Audio(X_amecro[1406], rate=sr)

In [ ]:
# ((299, 44100), (406, 44100)
# X_ameavo = get_bird_data(dataset_folder, 'ameavo', target_sr = 22050, chunk_seconds=2)
# X_amebit = get_bird_data('amebit', target_sr = 22050, chunk_seconds=2)

In [ ]:
from birds_utils import create_train_val_folders, create_train_val_folders_with_diff_files

In [ ]:
dataset_folder = './dataset/'
create_train_val_folders_with_diff_files(dataset_folder, ratio=0.2, all_subfolder='all/', train_subfolder='train/', val_subfolder='val/')

In [ ]:
all_subfolder = 'all/'
train_subfolder = 'train/'
val_subfolder = 'val/'

In [ ]:
dataset_folder_all = dataset_folder + all_subfolder
dataset_folder_train = dataset_folder + train_subfolder
dataset_folder_val = dataset_folder + val_subfolder

In [ ]:
len(glob(dataset_folder_train+'**/*', recursive=True))

In [ ]:
len(glob(dataset_folder_val+'**/*', recursive=True))

In [ ]:
len(glob(dataset_folder_all+'**/*', recursive=True))